In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, Reshape, LeakyReLU, Dropout, Embedding, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D,MaxPool2D
from keras.layers import  Dropout, Activation
from keras.optimizers import Adam, SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf
import glob
import random
import cv2
from random import shuffle
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from numpy import random
from tensorflow.keras.utils import to_categorical
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot
from keras import backend
from tensorflow.keras import datasets, layers, models
from keras import regularizers 
import pandas as pd

In [ ]:
!wget https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/rscbjbr9sj-3.zip

In [ ]:
!unzip rscbjbr9sj-3.zip 

In [ ]:
!unzip ZhangLabData.zip

In [ ]:
def load_files():
  base = 'CellData/OCT/train/'
  folders = [('NORMAL',0),('CNV',1),('DME',2),('DRUSEN',3)]  
  
  files = []
  for folder in folders:
    files_folder = os.listdir(f'{base}{folder[0]}')
    if folder[0] == 'NORMAL':
      files_folder = files_folder[0:26565]
    for i in range(len(files_folder)):
      files.append((f'{base}{folder[0]}/{files_folder[i]}',folder[1]))
  random.seed(4)
  random.shuffle(files)
  return files

In [ ]:
def load_images(files,parts=100,part=1):
  files_len = len(files)
  files_part = files[int((part-1)*(files_len/parts)):int((part)*(files_len/parts))]
  size=(224,224)
  images = []
  labales = []

  for file in files_part:
    raw = Image.open(file[0])
    raw = raw.resize(size)
    raw = np.array(raw)
    if(raw.shape == size):
      images.append(raw)
      labales.append(file[1])
      
  return np.array(images), np.array(labales)

In [ ]:
def load_images_percent(files,percent=1):
  files_len = len(files)
  percent_amount = int((files_len * percent)/100)
  point_split = int((files_len * 40)/100)
  files_part = files[point_split:point_split+percent_amount]
  size=(224,224)
  images = []
  labales = []

  for file in files_part:
    raw = Image.open(file[0])
    raw = raw.resize(size)
    raw = np.array(raw)
    if(raw.shape == size):
      images.append(raw)
      labales.append(file[1])
      
  return np.array(images), np.array(labales)

In [ ]:
def load_images_RAM(datax,datay,parts=100,part=1):
  files_len = len(datay)
  x_part = datax[int((part-1)*(files_len/parts)):int((part)*(files_len/parts))]
  y_part = datay[int((part-1)*(files_len/parts)):int((part)*(files_len/parts))]
  return x_part, y_part

In [ ]:
files = load_files()

In [ ]:
datax, datay = load_images(files,1,1)

In [ ]:
datax = datax[837:]
datay = datay[837:]

In [ ]:
train_x, train_y = load_images(files,100,1)

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(224,224,1), n_classes=4):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # scale up to image dimensions with linear activation
    n_nodes = in_shape[0] * in_shape[1]
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((in_shape[0], in_shape[1], 1))(li)
    # image input
    in_image = Input(shape=in_shape)
    # concat label as a channel
    merge = Concatenate()([in_image, li])
  

    ###################################################################
    # downsample to 112*112
    fe = Conv2D(32, (3,3), strides=(2,2), padding='same')(merge)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample to 56*56
    fe = Conv2D(64, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample to 28*28
    fe = Conv2D(64, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample to 14*14
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample to 7*7
    fe = Conv2D(256, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    ###################################################################


    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    fe = Dropout(0.4)(fe)
    # output
    out_layer = Dense(1, activation='sigmoid')(fe)
    # define model
    model = Model([in_image, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
# define the standalone generator model
def define_generator(latent_dim, n_classes=4):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # linear multiplication
    n_nodes = 7 * 7
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((7, 7, 1))(li)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    #########################################################
    # upsample to 14x14
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 28x28
    gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 56x56
    gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 112x112
    gen = Conv2DTranspose(32, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 224x224
    gen = Conv2DTranspose(16, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    #########################################################
    # output
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
    # define model
    model = Model([in_lat, in_label], out_layer)
    return model

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_label = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [ ]:
####Helper Functions####


def show_images2(gen):
    z_input, labels_input = generate_latent_points(100, 16)
    gen_imgs = gen.predict([z_input, labels_input])

    # gen_imgs = gen.predict(z)
    gen_imgs = 0.5*gen_imgs + 0.5

    fig,axs = plt.subplots(4,4,figsize=(4,4),sharey=True,sharex=True)

    cnt=0
    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
            axs[i, j].axis('off')
            cnt+=1

    fig.show()
    plt.show()

# load fashion mnist images
def load_real_samples(i,l):
    # load dataset
    # (trainX, trainy), (_, _) = load_data()
    trainX = i
    trainy = l
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    print(X.shape, trainy.shape)
    return [X, trainy]

# # select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=4):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y

def summarize_performance(step, g_model, c_model, latent_dim, dataset, n_samples=100):
    X, y = dataset
    _, acc = c_model.evaluate(X, y, verbose=0)
    print('Classifier preci: %.3f%%' % (acc * 100))
    show_images2(g_model)

def load_real_samples_test(a,b):
    # load dataset
    # (trainX, trainy), (_, _) = load_data()
    trainX = a
    trainy = b
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    print(X.shape, trainy.shape)
    return [X, trainy]


def calc_metrics(y_true, y_predi, Nclass=4 ):
  Precision = []
  F1 =  []
  Specifity =   []
  sensitivity =   []
  Accs=   []
  for c in range(Nclass):
      TP = np.sum( (y_true == c)&(y_predi==c) )
      FP = np.sum( (y_true != c)&(y_predi==c) )
      FN = np.sum( (y_true == c)&(y_predi != c)) 
      TN = np.sum( (y_true != c)&(y_predi != c)) 

      prec = TP/float(TP+FP)
      Precision.append(prec)
      f1= (2*TP)/float((2*TP) + FP + FN)
      F1.append(f1)
      ACC=(TP+TN)/float(TP+FN + TN + FP)
      Accs.append(ACC)
      SPE=TN/float(TN+FP)
      Specifity.append(SPE)
      SEN=TP/float(TP+FN)
      sensitivity.append(SEN)
  return Precision, sensitivity, Specifity, F1, Accs


def result_calc_RAM(classifier,amount=100):
  split_point = int(((100-amount) * len(datay)) / 100)
  prec_array = []
  accu_array = []
  sen_array = []
  sp_array = []
  fscore_array = []
  ytrue = []
  yped = []
  for i in range(100):
    print(i)
    atest,btest = load_images_RAM(datax[split_point:],datay[split_point:],100,i+1)
    x, y = load_real_samples_test(atest,btest)
    y = to_categorical(y,num_classes=8)
    y_true = np.argmax(y, axis=1)
    y_pred = classifier.predict(x).argmax(axis=1)
    ytrue.extend(y_true)
    yped.extend(y_pred)
    preci, sensitivity, sp, fscore, accu=calc_metrics(y_true, y_pred)
    prec_array.append(preci)
    sen_array.append(sensitivity)
    sp_array.append(sp)
    fscore_array.append(fscore)
    accu_array.append(accu)
  prec_array = np.array(prec_array).mean(axis=0)
  sen_array = np.array(sen_array).mean(axis=0)
  sp_array = np.array(sp_array).mean(axis=0)
  fscore_array = np.array(fscore_array).mean(axis=0)
  accu_array = np.array(accu_array).mean(axis=0)
  return {
      "precision": [prec_array,prec_array.mean()],
      "se": [sen_array,sen_array.mean()],
      "sp": [sp_array,sp_array.mean()],
      "f1": [fscore_array,fscore_array.mean()],
      "accu": [accu_array,accu_array.mean()],
  }, ytrue, yped

def result_calc(classifier):
  prec_array = []
  accu_array = []
  sen_array = []
  sp_array = []
  fscore_array = []
  for i in range(100):
    print(i)
    atest,btest = load_images(files,100,i+1)
    x, y = load_real_samples_test(atest,btest)
    y = to_categorical(y,num_classes=8)
    y_true = np.argmax(y, axis=1)
    y_pred = classifier.predict(x).argmax(axis=1)
    preci, sensitivity, sp, fscore, accu=calc_metrics(y_true, y_pred)
    prec_array.append(preci)
    sen_array.append(sensitivity)
    sp_array.append(sp)
    fscore_array.append(fscore)
    accu_array.append(accu)
  prec_array = np.array(prec_array).mean(axis=0)
  sen_array = np.array(sen_array).mean(axis=0)
  sp_array = np.array(sp_array).mean(axis=0)
  fscore_array = np.array(fscore_array).mean(axis=0)
  accu_array = np.array(accu_array).mean(axis=0)
  return {
      "precision": [prec_array,prec_array.mean()],
      "se": [sen_array,sen_array.mean()],
      "sp": [sp_array,sp_array.mean()],
      "f1": [fscore_array,fscore_array.mean()],
      "accu": [accu_array,accu_array.mean()],
  }

def cus_specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def shuffle_data(a,b):
    c = list(zip(a, b))
    random.shuffle(c)
    a, b = zip(*c)
    return np.array(a),np.array(b)

def select_imgs(number,numclass,gmodel):
  batch_number = int(number/numclass)
  result_imgse = []
  result_lbs = []
  for i in range(numclass):
    x_input = randn(100 * batch_number)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(batch_number, 100)
    # generate labels
    labels = np.array([i]*batch_number)
    gen_imgs = gmodel.predict([z_input, labels])
    result_imgse.extend(gen_imgs)
    result_lbs.extend([i]*batch_number)
  return np.array(result_imgse), np.array(result_lbs)

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10000, n_batch=128):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        print(i)
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
            # generate 'fake' examples
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
            # prepare points in latent space as input for the generator
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            # summarize loss on this batch
            # print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
            #   (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        if (i+1) % 10 == 0:
            print(i)
            show_images2(g_model)

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)

# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
x,y = load_images(files,100,1)
dataset = load_real_samples(x,y)

In [ ]:
train(g_model, d_model, gan_model, dataset, latent_dim)

In [ ]:
def define_classifier(in_shape=(224,224,1), n_classes=8):
    # image input
    in_image = Input(shape=in_shape)
    # downsample
    fe = Conv2D(128, (3,3), strides=(1,1), padding='same',activation='relu')(in_image)
    fe = MaxPool2D()(fe)
    fe = Dropout(0.4)(fe)

    # fe = LeakyReLU(alpha=0.2)(fe)

    fe = Conv2D(32, (3,3), strides=(2,2), padding='same',activation='relu')(fe)
    fe = MaxPool2D()(fe)
    fe = Dropout(0.4)(fe)

    fe = Conv2D(32, (3,3), strides=(1,1), padding='same',activation='relu')(fe)
    fe = MaxPool2D()(fe)
    fe = Dropout(0.4)(fe)

    # fe = LeakyReLU(alpha=0.2)(fe)
    
    fe = Conv2D(128, (3,3), strides=(1,1), padding='same',activation='relu')(fe)
    fe = MaxPool2D()(fe)
    fe = Dropout(0.4)(fe)


    fe = Conv2D(32, (3,3), strides=(1,1), padding='same',activation='relu')(fe)
    fe = MaxPool2D()(fe)
    fe = Dropout(0.4)(fe)


    fe = Conv2D(128, (3,3), strides=(2,2), padding='same',activation='relu')(fe)
    fe = MaxPool2D()(fe)
    fe = Dropout(0.4)(fe)

    

    # fe = LeakyReLU(alpha=0.2)(fe)



    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    # output layer nodes
    # fe = Dense(100)(fe)
    
    fe = Dropout(0.4)(fe)
    
    fe = Dense(n_classes)(fe)
    # supervised output
    c_out_layer = Activation('softmax')(fe)
    # define and compile supervised discriminator model
    c_model = Model(in_image, c_out_layer)
    # c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
    c_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy',tf.keras.metrics.Recall(), cus_specificity, f1_m ])
    return  c_model

In [ ]:
def classifier_1(in_shape=(224,224,1), n_classes=8):
  model = models.Sequential()
  model.add(layers.Conv2D(6, (7, 7), strides=(2,2), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None, input_shape=in_shape))
  # model.add(BatchNormalization())
  model.add(layers.PReLU())
  model.add(layers.Conv2D(16,(7,7), strides=(2,2), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  # model.add(BatchNormalization())
  model.add(layers.PReLU())
  model.add(layers.MaxPooling2D((2, 2), strides=2))
  model.add(layers.Dropout(0.25))

  model.add(layers.Conv2D(36,(5,5), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  # model.add(BatchNormalization())
  model.add(layers.PReLU())
  model.add(layers.Conv2D(36,(5,5), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  # model.add(BatchNormalization())
  model.add(layers.PReLU())
  model.add(layers.MaxPooling2D((2, 2), strides=2))
  model.add(layers.Dropout(0.25))

  model.add(layers.Conv2D(64,(3,3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  # model.add(BatchNormalization())
  model.add(layers.PReLU())
  model.add(layers.Dropout(0.25))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, kernel_initializer='he_normal', bias_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01) , activation=None))
  model.add(layers.PReLU())
  model.add(layers.Dropout(0.25))
  model.add(layers.Dense(n_classes, kernel_initializer='he_normal', bias_initializer='he_normal', activation='softmax'))
  # model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
  model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy',tf.keras.metrics.Recall(), cus_specificity, f1_m ])
  return  model

In [ ]:
def classifier_2(in_shape=(224,224,1), n_classes=8):
  model = models.Sequential()
  model.add(layers.Conv2D(64, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None, input_shape=in_shape))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.Conv2D(64, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.MaxPooling2D((3, 3), strides=2))

  model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal',  activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal',  activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.MaxPooling2D((3, 3), strides=2))

  model.add(layers.Flatten())
  model.add(layers.Dense(256, kernel_initializer='he_normal', bias_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01) , activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(256, kernel_initializer='he_normal', bias_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01) , activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(n_classes, kernel_initializer='he_normal', bias_initializer='he_normal', activation='softmax'))
  model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy',tf.keras.metrics.Recall(), cus_specificity, f1_m ])
  return  model

In [ ]:
def classifier_3(in_shape=(224,224,1), n_classes=8):
  model = models.Sequential()
  model.add(layers.Conv2D(64, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None, input_shape=in_shape))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.Conv2D(64, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.Conv2D(64, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.MaxPooling2D((3, 3), strides=2))

  model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal',  activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal',  activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal',  activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.MaxPooling2D((3, 3), strides=2))

  model.add(layers.Flatten())
  model.add(layers.Dense(256, kernel_initializer='he_normal', bias_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01) , activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(256, kernel_initializer='he_normal', bias_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01) , activation=None))
  model.add(layers.LeakyReLU(alpha=0.333))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(n_classes, kernel_initializer='he_normal', bias_initializer='he_normal', activation='softmax'))
  
  model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy',tf.keras.metrics.Recall(), cus_specificity, f1_m ])
  return  model

In [ ]:
def classifier_4(in_shape=(224,224,1), n_classes=8):
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None, input_shape=in_shape))
  model.add(layers.ReLU())
  # model.add(layers.Dropout(0.5))
  model.add(layers.Conv2D(64, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  model.add(layers.ReLU())
  # model.add(layers.Dropout(0.5))
  model.add(layers.MaxPooling2D((2, 2), strides=2))
  # model.add(BatchNormalization())
  # model.add(layers.Dropout(0.5))

  model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  model.add(layers.ReLU())
  # model.add(layers.Dropout(0.5))
  model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  model.add(layers.ReLU())
  # model.add(layers.Dropout(0.5))
  model.add(layers.MaxPooling2D((2, 2), strides=2))
  # model.add(BatchNormalization())
  # model.add(layers.Dropout(0.5))

  model.add(layers.Flatten())
  model.add(layers.Dense(256, kernel_initializer='he_normal', bias_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01) , activation=None))
  model.add(layers.ReLU())
  model.add(layers.Dense(n_classes, kernel_initializer='he_normal', bias_initializer='he_normal', activation='softmax'))
  model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy',tf.keras.metrics.Recall(), cus_specificity, f1_m ])
  return  model

In [ ]:
def classifier_5(in_shape=(224,224,1), n_classes=8):
  model = models.Sequential()
  model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None, input_shape=in_shape))
  model.add(layers.ReLU())
  model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  model.add(layers.ReLU())
  model.add(layers.MaxPooling2D((2, 2), strides=2))

  model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  model.add(layers.ReLU())
  model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding='same', data_format="channels_last", kernel_initializer='he_normal', bias_initializer='he_normal', activation=None))
  model.add(layers.ReLU())

  model.add(layers.Flatten())
  model.add(layers.Dense(512, kernel_initializer='he_normal', bias_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01) , activation=None))
  model.add(layers.ReLU())
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(512, kernel_initializer='he_normal', bias_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01) , activation=None))
  model.add(layers.ReLU())
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(n_classes, kernel_initializer='he_normal', bias_initializer='he_normal', activation='softmax'))
  model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy',tf.keras.metrics.Recall(), cus_specificity, f1_m ])
  return  model

In [ ]:
classifier = define_classifier()
# classifier = classifier_1()

# print(classifier.summary())

# x,y = load_images(files,30,15)
imgs,labels = load_images_percent(files,20)
imgs, labels = load_real_samples(x,y)

labels = to_categorical(labels,num_classes=8)
training = classifier.fit(x=imgs,y=labels,batch_size=64,epochs=50,verbose=1)    

In [ ]:
gen_imgs , pseudo_labels = select_imgs(imgs.shape[0],4,g_model)
pseudo_labels += 4
pseudo_labels = to_categorical(pseudo_labels,num_classes=8)

In [ ]:
whole_imgs = np.append(gen_imgs, imgs,axis=0)
whole_labels = np.append(pseudo_labels, labels,axis=0)
whole_imgs,whole_labels =  shuffle_data(list(whole_imgs),list(whole_labels))
classifier.fit(x=whole_imgs,y=whole_labels ,batch_size=64,epochs=200,verbose=1)

In [ ]:
loss_array = []
acc_array = []
sen_array = []
sp_array = []
fscore_array = []
for i in range(100):
  print(i)
  atest,btest = load_images(files,100,i+1)
  x, y = load_real_samples_test(atest,btest)
  y = to_categorical(y,num_classes=8)
  loss, accuracy, sensitivity, sp, fscore = classifier.evaluate(x,y)
  loss_array.append(loss)
  acc_array.append(accuracy)
  sen_array.append(sensitivity)
  sp_array.append(sp)
  fscore_array.append(fscore)


print("accuracy",np.array(acc_array).mean())
print("se",np.array(sen_array).mean())
print("sp",np.array(sp_array).mean())
print("f_score",np.array(fscore_array).mean()) 

In [ ]:
loss_array = []
acc_array = []
sen_array = []
sp_array = []
fscore_array = []
for i in range(100):
  print(i)
  atest,btest = load_images_RAM(datax,datay,100,i+1)
  x, y = load_real_samples_test(atest,btest)
  y = to_categorical(y,num_classes=8)
  loss, accuracy, sensitivity, sp, fscore = classifier.evaluate(x,y)
  loss_array.append(loss)
  acc_array.append(accuracy)
  sen_array.append(sensitivity)
  sp_array.append(sp)
  fscore_array.append(fscore)


print("accuracy",np.array(acc_array).mean())
print("se",np.array(sen_array).mean())
print("sp",np.array(sp_array).mean())
print("f_score",np.array(fscore_array).mean()) 

In [ ]:
#20_150
np.savetxt("myc_20.csv", np.array([yt, yp]), delimiter=",")

In [ ]:
result

In [ ]:
result,yt,yp = result_calc_RAM(classifier,80)

In [ ]:
result